In [9]:
%load_ext autoreload
%autoreload 2
import os
import copy
import pickle
import pandas as pd
import copy
from statistics import mean

import numpy as np
import scipy
from tqdm import tqdm

from gensim.models import Word2Vec
from gensim import utils
from gensim.corpora import Dictionary
from gensim.models.lsimodel import LsiModel
from sklearn.preprocessing import MinMaxScaler

import sys
sys.path.append('../src')
from models import get_matrices_from_term_lists, produce_2ndorder_effect_size, produce_1storder_effect_size_unnormalized
from models import produce_2ndorder_p_value, produce_1storder_p_value


In [10]:
we_model_name = "sg_dim300_min100_win5"
we_vector_size = 300
we_model_dir = '../data/external/wiki-english/wiki-english-20171001/%s' % we_model_name

In [11]:
we_model = Word2Vec.load(we_model_dir+'/model.gensim')

print ('loading done!')

loading done!


In [12]:
#loading E_ctx_vec
with open(we_model_dir+'/E_ctx_vec.pkl', 'rb') as fr:
    E_ctx_vec = pickle.load(fr)
with open(we_model_dir+'/E_wrd_vec.pkl', 'rb') as fr:
    E_wrd_vec = pickle.load(fr)

# Calculating *d*

In [16]:
# WEAT 1
X_terms = ['aster', 'clover', 'hyacinth', 'marigold', 'poppy', 'azalea', 
           'crocus', 'iris', 'orchid', 'rose', 'bluebell', 'daffodil','lilac',
           'pansy','tulip','buttercup','daisy','lily','penny','violet','carnation',
           'magnolia','petunia','zinnia']
Y_terms = ['caterpillar','flea','locust','spider','bedbug','centipede','fly',
          'maggot','tarantula','bee','cockroach','gnat','mosquito','termite','beetle',
          'cricket','hornet','moth','wasp','blackfly','dragonfly','horsefly','roach',
          'weevil']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'divorce','jail','poverty','ugly','cancer','kill','rotten','vomit','agony',
          'prison']
# Removed words:
# X: gladiola (not found)
# Y: ant (random)

In [18]:
# WEAT 2
X_terms = ['bagpipe','cello','guitar','lute','trombone','banjo','clarinet','harmonica',
           'mandolin','trumpet','bassoon','drum','harp','oboe','tuba','bell','fiddle',
           'harpsichord','piano','viola','bongo','flute','horn','saxophone']
Y_terms = ['arrow','club','gun','missile','spear','axe','dagger','harpoon','pistol',
          'sword','blade','dynamite','hatchet','rifle','tank','bomb','firearm',
          'knife','shotgun','teargas','cannon','grenade','mace','slingshot','whip']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'divorce','jail','poverty','ugly','cancer','kill','rotten','vomit','agony',
          'prison']

In [34]:
# WEAT 3
X_terms = ['roger','alan','frank','ian','justin',
          'ryan','andrew','fred','jack','matthew','stephen','brad','greg','jed',
          'paul','todd','brandon','hank','jonathan','peter','wilbur','amanda',
          'courtney','heather','melanie','sara','amber','crystal','katie',
          'meredith','shannon','betsy','donna','kristin','nancy','stephanie',
          'bobbie-sue','ellen','lauren','peggy','sue-ellen','colleen','emily',
          'megan','rachel','wendy']
Y_terms = ['alonzo','jamel','lerone','theo','alphonse','jerome','leroy',
           'torrance','darnell','lamar','lionel','rashaun','tyree','deion',
          'lamont','malik','terrence','tyrone','lavon','terryl',
          'wardell','aiesha','lashelle','nichelle','shereen','temeka','ebony',
          'latisha','shaniqua','tameisha','teretha','jasmine','laronya','shanise',
          'tanisha','tia','lakisha','latoya','sharise','tashika','yolanda',
          'lashandra','malika','shavonn','tawanda','yvette']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
          'loyal','pleasure','diamond','gentle','honest','lucky','rainbow',
          'diploma','gift','honor','miracle','sunrise','family','happy','laughter',
          'paradise','vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy','bomb',
          'divorce','jail','poverty','ugly','cancer','evil','kill','rotten','vomit']
print ("Missing x terms: ", [x for x in X_terms if x not in we_model.wv])
print('missing y terms: ', [x for x in Y_terms if x not in we_model.wv])
# Removed terms
# X: adam (random), chip (random), harry (random), josh (random)
# Y: percell (not found), rasaan (not found), everol (not found), marcelius (not found)

Missing x terms:  ['bobbie-sue', 'sue-ellen']
missing y terms:  ['terryl', 'aiesha', 'lashelle', 'temeka', 'tameisha', 'teretha', 'laronya', 'shanise', 'lakisha', 'sharise', 'tashika', 'lashandra', 'shavonn']


In [43]:
# WEAT 4
X_terms = ['Brad', 'Brendan', 'Geoffrey', 'Greg', 'Brett', 'Jay', 'Matthew', 'Neil',
'Todd', 'Allison', 'Anne', 'Carrie', 'Emily', 'Jill', 'Laurie', 'Kristen', 'Meredith',
           'Sarah']
Y_terms = ['Darnell', 'Hakim', 'Jermaine', 'Kareem', 'Jamal', 'Leroy', 'Rasheed',
'Tremayne', 'Tyrone', 'Aisha', 'Ebony', 'Keisha', 'Kenya', 'Latonya', 'Latoya',
           'Tamika', 'Tanisha']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'bomb','divorce','jail','poverty','ugly','cancer','evil','kill',
          'rotten','vomit']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]


In [45]:
# WEAT 5
X_terms = ['Brad', 'Brendan', 'Geoffrey', 'Greg', 'Brett', 'Jay', 'Matthew', 'Neil',
'Todd', 'Allison', 'Anne', 'Carrie', 'Emily', 'Jill', 'Laurie', 'Kristen', 'Meredith',
           'Sarah']
Y_terms = ['Darnell', 'Hakim', 'Jermaine', 'Kareem', 'Jamal', 'Leroy', 'Rasheed',
'Tremayne', 'Tyrone', 'Aisha', 'Ebony', 'Keisha', 'Kenya', 'Latonya', 'Latoya',
           'Tamika', 'Tanisha']
A_terms = ['joy', 'love', 'peace', 'wonderful', 'pleasure', 'friend', 'laughter', 'happy']
B_terms = ['agony', 'terrible', 'horrible', 'nasty', 'evil', 'war', 'awful', 'failure']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]
# Removed terms: "Lakisha"

In [5]:
# WEAT 6
X_terms = ['John', 'Paul','Mike','Kevin','Steve','Greg','Jeff','Bill']
Y_terms = ['Amy','Joan','Lisa','Sarah','Diana','Kate','Ann','Donna']
A_terms = ['executive','management','professional','corporation',
               'salary','office','business','career']
B_terms = ['home','parents','children','family',
               'cousins','marriage','wedding','relatives']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [13]:
# WEAT 7 using Google News word2vec model
X_terms = ['math','algebra','geometry','calculus',
             'equations','computation','numbers','addition']
Y_terms = ['poetry','art','dance','literature',
             'novel','symphony','drama','sculpture']
A_terms = ['male','man','boy','brother',
              'he','him','his','son']
B_terms = ['female','woman','girl','sister',
               'she','her','hers','daughter']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [33]:
# Weat 8
A_terms = ['brother','father','uncle','grandfather',
          'son','he','his','him']
B_terms = ['sister','mother','aunt','grandmother',
          'daughter','she','hers','her']
X_terms = ['science','technology','physics','chemistry',
          'Einstein','NASA','experiment','astronomy']
Y_terms = ['poetry','art','Shakespeare','dance',
          'literature','novel','symphony','drama']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [35]:
# WEAT 9
X_terms = ['sad','hopeless','gloomy','tearful','miserable','depressed']
Y_terms = ['sick','illness','influenza','disease','virus','cancer']
A_terms = ['impermanent','unstable','variable','fleeting','short','brief',
          'occasional']
B_terms = ['stable','always','constant','persistent','chronic','prolonged','forever']

In [7]:
# Weat 10
X_terms = ['Tiffany', 'Michelle', 'Cindy', 'Kristy', 'Brad', 'Eric', 'Joey', 'Billy']
Y_terms = ['Ethel', 'Bernice', 'Gertrude', 'Agnes', 'Cecil', 'Wilbert', 'Mortimer', 'Edgar']
A_terms = ['joy', 'love', 'peace', 'wonderful', 'pleasure', 'friend', 'laughter', 'happy']
B_terms = ['agony', 'terrible', 'horrible', 'nasty', 'evil', 'war', 'awful', 'failure']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [10]:
produce_1storder_effect_size_unnormalized(X_terms, Y_terms, A_terms, B_terms, we_model, E_ctx_vec, E_wrd_vec)

1.815586923924345

# First-Order Experiments

In [46]:
print(produce_1storder_effect_size_unnormalized(X_terms, Y_terms, A_terms, B_terms, we_model, E_ctx_vec, E_wrd_vec))
print(X_terms)

0.2104150295245534
['brad', 'brendan', 'geoffrey', 'greg', 'brett', 'jay', 'matthew', 'neil', 'todd', 'allison', 'anne', 'carrie', 'emily', 'jill', 'laurie', 'kristen', 'meredith', 'sarah']


In [ ]:
print(produce_1storder_p_value(we_model, X_terms, Y_terms, A_terms, B_terms, E_ctx_vec, E_wrd_vec))
print(X_terms)

  0%|                                                           | 95/4601020896.932353 [03:54<3086558:33:38,  2.42s/it]

# Second-Order

In [26]:
produce_effect_size(we_model, X_terms, Y_terms, A_terms, B_terms)

1.5948170825435086

In [14]:
print(produce_2ndorder_p_value(we_model, X_terms, Y_terms, A_terms, B_terms))
print(X_terms)

100%|██████████████████████████████████████████████████████████████████████████| 12870/12870.0 [08:53<00:00, 24.12it/s]


0.0007125704370340991
['math', 'algebra', 'geometry', 'calculus', 'equations', 'computation', 'numbers', 'addition']


Results: 1.774, 1.595, 1.4183

# Normalized

In [78]:
def get_1storder_association_metric_scaled(word, A_terms, B_terms):
    A_relations = get_expSG_1storder_relation_scaled(word, A_terms)
    B_relations = get_expSG_1storder_relation_scaled(word, B_terms)
    return mean(A_relations.values()) - mean(B_relations.values())

In [92]:
def produce_1storder_effect_size_scaled(X_terms, Y_terms, A_terms, B_terms):
    x_associations = np.array([])
    y_associations = np.array([])
    for (x,y) in zip(X_terms, Y_terms):
        x_association = get_1storder_association_metric_scaled(x, A_terms, B_terms)
        y_association = get_1storder_association_metric_scaled(y, A_terms, B_terms)
        x_associations = np.append(x_associations, x_association)
        y_associations = np.append(y_associations, y_association)
    all_associations = np.append(x_associations, y_associations)
    return (np.mean(x_associations) - np.mean(y_associations))/np.std(all_associations, ddof=1)
produce_1storder_effect_size_scaled(X_terms, Y_terms, A_terms, B_terms)

1.4517385120046582

(2nd order, 1st order unscaled, 1st order scaled)
(1.774, 1.816, 1.759)
(1.595, 1.648, 1.335)
(1.4183, 1.471, 1.451)